In [1]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

import os 
import json 
import urllib
from urllib.parse import unquote


In [ ]:

def kpop_crawler_url(kpop_url_list=dict()):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    for num in range(1,6):
        
        url = f'https://korean.visitseoul.net/hallyu?curPage={num}&srchType=&srchOptnCode=&srchCtgry=29&sortOrder=&srchWord='
        driver.get(url)
        print(url)

        try:
            article_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
            )
            articles = article_list.find_elements_by_tag_name("li")
            for article in articles:
                a_tag = article.find_element_by_tag_name("a")

                title = a_tag.get_attribute('title')
                href  = a_tag.get_attribute('href')
                href = urllib.parse.unquote(href)
                thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

                kpop_url_list[title] = {
                    'href': href,
                    'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
                }
                print(title)
                print(href)
                print(thumb)

        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex) #
            driver.quit()

    driver.quit()
    return kpop_url_list

In [2]:

def korea_pop_detail_page(url):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    # url = 'https://korean.visitseoul.net/hallyu/근대역사-골목산책-우리슈퍼-코스_/34204'
    driver.get(url)
    try:
        info_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'infor-element'))
        )
        category_title_selenium = info_element.find_element_by_class_name('h3')
        category_title = category_title_selenium.text
        print(category_title)

        text_area = info_element.find_element_by_class_name('text-area')
        tourist_spots = text_area.find_elements_by_class_name('cnt-blockquote')
        for spot in tourist_spots:
            spot_name = spot.find_element_by_class_name('fc-blue').text
            spot_info = spot.find_elements_by_tag_name('p')
            if '탐방코스' == spot_name:
                continue
            else:
                try:
                    description = '없음'
                    address = '없음'
                    open_hour = '없음'
                    tel = '없음'
                    website = '없음'
                    trans = '없음'
                    for infomation in spot_info:



                        if '주소' in infomation.text:
                            try:
                                address = infomation.text.strip().replace("주소: ", "")
                            except Exception as ex:
                                print('address error', ex)
                        elif '운영시간' in infomation.text:
                            try:
                                open_raw = infomation.text.strip()
                                open_hour = infomation.text.strip().replace("운영시간 :","").replace("운영시간:", "").replace("운영시간","")
                            except Exception as ex:
                                print('open_hour error', ex)
                                
                            
                        elif '전화번호' in infomation.text or '문의' in infomation.text:
                            
                            try:
                                tel = infomation.text.strip().replace("문의 :","").replace("문의:", "").replace("전화번호:", "").replace("전화번호 :", "")
                                tel = re.search('\d{2,3}-\d{3,4}-\d{3,4}', tel)
                                if tel == None:
                                    tel = re.search(' \d{4}-\d{4}', tel)
                                tel = tel.group()
                            except Exception as ex:
                                print('tel error', ex)
                                
                        elif '홈페이지' in infomation.text or '웹사이트' in infomation.text or '웹페이지' in infomation.text:
                            if '웹페이지' in infomation.text:
                                try:
                                    a_tag = infomation.find_element_by_tag_name('a')
                                    href = a_tag.get_attribute('href')
                                    href = urllib.parse.unquote(href)
                                    website = href
                                except Exception as ex:
                                    print('a tag href ::::::', ex)
                            else:
                                try:
                                    website = infomation.text.strip().replace("홈페이지: ", "").replace("웹사이트: ", "")
                                except Exception as ex:
                                    print('website error', ex)  

                        elif '교통편' in infomation.text:
                            try:
                                trans = infomation.text.strip().split(':')[1]
                            except Exception as ex:
                                print('trans error', ex)  
                        elif len(infomation.text) > 150:
                            try:
                                description = infomation.text
                            except Exception as ex:
                                print('description error', ex)                              
                        else:
                            text = ''

                except Exception as ex:
                    print('not iteral', ex)
            if '-' in spot_name:
                try:
                    spot_name = spot_name.split('-')[1].replace(" ", '')
                except Exception as ex:
                    print('여기서 나는 건가?', ex)
            print('placename : ' ,spot_name)
            print('description : ' , description)
            print('place address : ' ,address)
            print('place telphone : ' ,tel)
            print('place open_hour : ' ,open_hour)                                
            print('place website : ' ,website)
            print('place transportation : ' ,trans)
            place,_ = Place.objects.get_or_create(
                    name = spot_name,
                    content = description,
                    address = address,
                    phone_number = tel,
                    open_time = open_hour,
                    url = website,
                    trans = trans,

                )
            print('models make:::::::::::::::::::: ', place)
            kpopcontent, _ = KPopContent.objects.get_or_create(
                place = place,
                content_type = 'tv',
                title = category_title,
            )
            print('kpopcontent:::::::::::::::::::: ', kpopcontent)
            print('=========================================================')
            
    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) #
        driver.quit()
    driver.quit()

In [ ]:
kpop_url_list = kpop_crawler_url()
# print(kpop_url_list)
with open('./k_pop_url_page_fron_1_to_6_0801.json', 'w', encoding='UTF-8') as outfile:
    json.dump(kpop_url_list, outfile, ensure_ascii=False)

In [3]:
# k_pop_url_page_fron_1_to_6
with open('./k_pop_url_page_fron_1_to_6_0801.json', 'r') as file:
    json_data = json.load(file)
for title in json_data:
    url = json_data[title]['href']
    print(url)
    try:
        data = korea_pop_detail_page(url)
        
    except:
        print('다른 페이지 오류를 어떻게 잡지')
    

https://korean.visitseoul.net/hallyu/세븐틴과-함께하는-Left-Right-서울-여행_/34537


/home/kimdooh/.pyenv/versions/kopa-tour-env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


세븐틴과 함께하는 ‘Left & Right’ 서울 여행
placename :  크로바볼링장
description :  없음
place address :  서울 금천구 시흥대로 440
place telphone :  02-855-0080
place open_hour :   평일 11:00~03:00, 주말 10:00~03:00
place website :  없음
place transportation :   전철 1호선 독산역(#P143) 1번 출구에서 도보 20분
models make::::::::::::::::::::  크로바볼링장
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 세븐틴과 함께하는 ‘Left & Right’ 서울 여행 입니다
placename :  곱창 파는 고깃집
description :  없음
place address :  서울 마포구 마포대로 173-14(4층)
place telphone :  02-365-9245
place open_hour :   매일 16:00~24:00
place website :    www.instagram.com/hannah3659245
place transportation :   지하철 5호선 애오개역(#530) 1번 출구에서 도보 5분
models make::::::::::::::::::::  곱창 파는 고깃집
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 세븐틴과 함께하는 ‘Left & Right’ 서울 여행 입니다
placename :  다크룸포토매틱
description :  없음
place address :  서울 강남구 압구정로4길 27-2
place telphone :  070-4230-2019
place open_hour :   매일 14:00~21:00
place website :    www.darkroomseoul.co.kr
place transportation :   지하철 3호선 신사역(#337) 8번 출구에서 도보 

placename :  샤로스톤 외대점
description :  없음
place address :  주소 : 서울 동대문구 휘경로2가길 5
place telphone :  02-966-0867
place open_hour :   월~금요일 11:00~22:00, 토~일요일 11:30~22:00 (라스트 오더 21:00)
place website :  없음
place transportation :   지하철 1호선 외대앞역(#122) 6번 출구 도보 3분
models make::::::::::::::::::::  샤로스톤 외대점
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 엑소를 더 가까이, ‘엑소엘 투어’ 입니다
https://korean.visitseoul.net/hallyu/몬스타엑스의-에너지원-서울-맛집을-찾아서_/30399
몬스타엑스의 에너지원! 서울 맛집을 찾아서
placename :  마라토끼
description :  없음
place address :  서울 용산구 이태원로23길 16
place telphone :  없음
place open_hour :   일~목요일 11:30~23:30, 금~토요일 11:00~02:00
place website :   maratuji.modoo.at
place transportation :   지하철 6호선 이태원역(#630) 1번 출구에서 도보 2분
models make::::::::::::::::::::  마라토끼
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 몬스타엑스의 에너지원! 서울 맛집을 찾아서 입니다
placename :  모단걸응접실 가로수길점
description :  없음
place address :  서울 강남구 강남대로152길 62
place telphone :  02-3448-0815
place open_hour :   매일 11:30~22:00
place website :   moderngirl.fordining.

placename :  볼링볼링
description :  없음
place address :  서울 중구 청계천로 400 롯데캐슬 베니치아 B1F
place telphone :  02-3298-6606
place open_hour :   매일 24시간 운영
place website :  없음
place transportation :   지하철 1호선(#127), 6호선(#636) 동묘앞역 도보 10분
models make::::::::::::::::::::  볼링볼링
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 아이즈원을 따라 떠나는 서울 여행 입니다
placename :  파라솔분식
description :  없음
place address :  서울 노원구 상계로 1길 14-17
place telphone :  02-951-6128
place open_hour :   매일 11:00~21:00, 1월 1일 휴무
place website :  없음
place transportation :   지하철 4호선(#411), 7호선(#713) 노원역 9번 출구 도보 5분
models make::::::::::::::::::::  파라솔분식
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 아이즈원을 따라 떠나는 서울 여행 입니다
https://korean.visitseoul.net/hallyu/뉴이스트를-향한-사랑을-담은-러브-투어-2탄_/29555
뉴이스트를 향한 사랑을 담은 '러브 투어' 2탄
placename :  어반소스
description :  없음
place address :  서울 성동구 연무장 3길 9
place telphone :  02-462-6262
place open_hour :   카페 매일 11:00~23:00 (last order 22:30), 레스토랑 매일 11:30~21:30 (last order 21:00)
place website :  www.instagram.c

placename :  마크레인커피
description :  없음
place address :  서울 강남구 선릉로157길 23-5
place telphone :  02-516-5202
place open_hour :   월-토 10:00~23:00, 일 10:00~22:00
place website :  웹사이트 보기
place transportation :  없음
models make::::::::::::::::::::  마크레인커피
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 갓세븐의 서울 라이프 입니다
placename :  응봉산 팔각정
description :  없음
place address :  서울 성동구 응봉동 271
place telphone :  02-2286-6061
place open_hour :   연중무휴 00:00~24:00
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  응봉산 팔각정
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 갓세븐의 서울 라이프 입니다
placename :  코끼리볼링장
description :  없음
place address :  서울 중구 퇴계로 166 흥국빌딩
place telphone :  02-2268-6073
place open_hour :  월~목 10:30~02:00, 금~토 10:30~03:00, 일 10:30~24:00
place website :  웹사이트 보기  (한국어)
place transportation :  없음
models make::::::::::::::::::::  코끼리볼링장
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 갓세븐의 서울 라이프 입니다
https://korean.visitseoul.net/hallyu/NCT의-HotYoung-서울여행_/27242
NCT의 

placename :  카페 앤게더
description :  없음
place address :  서울특별시 용산구 독서당로 39 한남동신성미소시티
place telphone :  02-511-6220
place open_hour :   평일 08:00~21:00 주말, 공휴일 11:00~19:00 명절당일 휴무
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  카페 앤게더
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘아미’들을 위한 특별한 서울 여행 입니다
placename :  그 집
description :  없음
place address :  서울특별시 서초구 강남대로34길 46 서진빌딩
place telphone :  02-574-8548
place open_hour :   월~토 17:00~02:00, 일요일 휴무
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  그 집
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘아미’들을 위한 특별한 서울 여행 입니다
placename :  아차산
description :  없음
place address :  서울특별시 광진구 광장동 아차산
place telphone :  02-450-1655
place open_hour :   연중무휴
place website :  웹사이트 보기
place transportation :  없음
models make::::::::::::::::::::  아차산
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘아미’들을 위한 특별한 서울 여행 입니다
https://korean.visitseoul.net/hallyu/뉴이스트를-향한-사랑을-담은-러브-투어-코스_/25733
뉴이스트를 향한

placename :  오리올
description :  없음
place address :  주소 ： 서울 용산구 신흥로20길 43
place telphone :  02-6406-5252
place open_hour :   ： 브런치카페 11:30~18:00 바&다이닝18:00~(익일) 01:00
place website :  홈페이지 ： http://instagram.com/oriole_bar/
place transportation :  없음
models make::::::::::::::::::::  오리올
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘YG 삼총사’ 산다라박, 이하이, 이수현이 소개하는 이태원 명소 탐방 입니다
placename :  테일러블포우먼
description :  없음
place address :  주소 ： 서울 용산구 대사관로5길 33
place telphone :  02-749-7801
place open_hour :  없음
place website :  홈페이지 : http://blog.naver.com/buls13
place transportation :  없음
models make::::::::::::::::::::  테일러블포우먼
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘YG 삼총사’ 산다라박, 이하이, 이수현이 소개하는 이태원 명소 탐방 입니다
placename :  옹느세자메
description :  없음
place address :  주소 ： 서울 용산구 이태원로 54길 51
place telphone :  02-794-3446
place open_hour :   ：매일 11:00-22:00
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  옹느세자메
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 ‘YG 삼

placename :  더민스
description :  없음
place address :  주소 ： 서울 강남구 압구정로330 1층 112호
place telphone :  02-512-1452
place open_hour :   ： 10:00~19:00 연중무휴
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  더민스
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 지금은 BTS 시대! ‘방탄소년단’이 자주 출몰하는 서울 명소 탐방 입니다
placename :  지아니스나폴리 가로수길점
description :  없음
place address :  주소 ： 서울 강남구 도산대로 15길 15
place telphone :  02-3416-0316
place open_hour :  없음
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  지아니스나폴리 가로수길점
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 지금은 BTS 시대! ‘방탄소년단’이 자주 출몰하는 서울 명소 탐방 입니다
placename :  함경도찹쌀순대
description :  없음
place address :  주소 ： 서울 송파구 송파대로28길 32
place telphone :  02-403-8822
place open_hour :   ：00:00~24:00 연중무휴
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  함경도찹쌀순대
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 지금은 BTS 시대! ‘방탄소년단’이 자주 출몰하는 서울 명소 탐방 입니다
https://korean.visitseoul.net/hally

placename :  너드
description :  없음
place address :  주소 : 서울 용산구 이태원로17길 22
place telphone :  02-798-2221
place open_hour :   18:00 ~ 02:00
place website :  없음
place transportation :  없음
models make::::::::::::::::::::  너드
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 TV 밖을 나온 스타, 맛집 오너가 되다! 입니다
https://korean.visitseoul.net/hallyu/K-POP-아이돌의-가족이-운영하는-서울의-맛집_/18719
K-POP 아이돌의 가족이 운영하는 서울의 맛집!
placename :  BBQ 합정스타점
description :  없음
place address :  주소 ： 서울 마포구 양화로8길 5
place telphone :  050-7526-2126
place open_hour :   ： 11:30 ~익일 02:30
place website :  홈페이지 ： 11370.mbym.co.kr
place transportation :  없음
models make::::::::::::::::::::  BBQ 합정스타점
kpopcontent::::::::::::::::::::  카테고리 tv의 제목은 K-POP 아이돌의 가족이 운영하는 서울의 맛집! 입니다
placename :  BHC 사당점
description :  없음
place address :  주소 ： 서울 동작구 동작대로7길 16
place telphone :  050-4109-2005
place open_hour :   ： 12:00 ~ 24:00
place website :  홈페이지 ： www.bhc.co.kr
place transportation :  없음
models make::::::::::::::::::::  BHC 사당점
kpopcontent::::::

In [ ]:

def drama_crawler_url(kpop_url_list=dict()):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    for num in range(1,6):
        
        url = f'https://korean.visitseoul.net/hallyu?curPage={num}&srchType=&srchOptnCode=&srchCtgry=30&sortOrder=&srchWord='
        driver.get(url)
        print(url)

        try:
            article_list = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'article-list-slide'))
            )
            articles = article_list.find_elements_by_tag_name("li")
            for article in articles:
                a_tag = article.find_element_by_tag_name("a")

                title = a_tag.get_attribute('title')
                href  = a_tag.get_attribute('href')
                href = urllib.parse.unquote(href)
                thumb = a_tag.find_element_by_class_name('thumb').get_attribute('style')

                kpop_url_list[title] = {
                    'href': href,
                    'thumb' : 'https://korean.visitseoul.net/' +thumb.replace('background-image: url("','').replace('");',''),
                }
                print(title)
                print(href)
                print(thumb)

        except Exception as ex: # 에러 종류
            print('에러가 발생 했습니다', ex) #
            driver.quit()

    driver.quit()
    return kpop_url_list

In [ ]:

def korea_drama_detail_page(url):
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    driver = webdriver.chrome.webdriver.WebDriver(executable_path='../chromedriver', chrome_options=options)
    # url = 'https://korean.visitseoul.net/hallyu/근대역사-골목산책-우리슈퍼-코스_/34204'
    driver.get(url)
    try:
        info_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'infor-element'))
        )
        category_title_selenium = info_element.find_element_by_class_name('h3')
        category_title = category_title_selenium.text
        print(category_title)

        text_area = info_element.find_element_by_class_name('text-area')
        tourist_spots = text_area.find_elements_by_class_name('cnt-blockquote')
        for spot in tourist_spots:
            spot_name = spot.find_element_by_class_name('fc-blue').text
            spot_info = spot.find_elements_by_tag_name('p')
            if '탐방코스' == spot_name:
                continue
            else:
                try:
                    description = '없음'
                    address = '없음'
                    open_hour = '없음'
                    tel = '없음'
                    website = '없음'
                    trans = '없음'
                    for infomation in spot_info:



                        if '주소' in infomation.text:
                            try:
                                address = infomation.text.strip().replace("주소: ", "")
                            except Exception as ex:
                                print('address error', ex)
                        elif '운영시간' in infomation.text:
                            try:
                                open_raw = infomation.text.strip()
                                open_hour = infomation.text.strip().replace("운영시간 :","").replace("운영시간:", "").replace("운영시간","")
                            except Exception as ex:
                                print('open_hour error', ex)
                                
                            
                        elif '전화번호' in infomation.text or '문의' in infomation.text:
                            
                            try:
                                tel = infomation.text.strip().replace("문의 :","").replace("문의:", "").replace("전화번호:", "").replace("전화번호 :", "")
                                tel = re.search('\d{2,3}-\d{3,4}-\d{3,4}', tel)
                                if tel == None:
                                    tel = re.search(' \d{4}-\d{4}', tel)
                                tel = tel.group()
                            except Exception as ex:
                                print('tel error', ex)
                                
                        elif '홈페이지' in infomation.text or '웹사이트' in infomation.text or '웹페이지' in infomation.text:
                            if '웹페이지' in infomation.text:
                                try:
                                    a_tag = infomation.find_element_by_tag_name('a')
                                    href = a_tag.get_attribute('href')
                                    href = urllib.parse.unquote(href)
                                    website = href
                                except Exception as ex:
                                    print('a tag href ::::::', ex)
                            else:
                                try:
                                    website = infomation.text.strip().replace("홈페이지: ", "").replace("웹사이트: ", "")
                                except Exception as ex:
                                    print('website error', ex)  

                        elif '교통편' in infomation.text:
                            try:
                                trans = infomation.text.strip().split(':')[1]
                            except Exception as ex:
                                print('trans error', ex)  
                        elif len(infomation.text) > 150:
                            try:
                                description = infomation.text
                            except Exception as ex:
                                print('description error', ex)                              
                        else:
                            text = ''

                except Exception as ex:
                    print('not iteral', ex)
            if '-' in spot_name:
                try:
                    spot_name = spot_name.split('-')[1].replace(" ", '')
                except Exception as ex:
                    print('여기서 나는 건가?', ex)
            print('placename : ' ,spot_name)
            print('description : ' , description)
            print('place address : ' ,address)
            print('place telphone : ' ,tel)
            print('place open_hour : ' ,open_hour)                                
            print('place website : ' ,website)
            print('place transportation : ' ,trans)
            place,_ = Place.objects.get_or_create(
                    name = spot_name,
                    content = description,
                    address = address,
                    phone_number = tel,
                    open_time = open_hour,
                    url = website,
                    trans = trans,

                )
            print('models make:::::::::::::::::::: ', place)
            kpopcontent, _ = KPopContent.objects.get_or_create(
                place = place,
                content_type = 'tv',
                title = category_title,
            )
            print('kpopcontent:::::::::::::::::::: ', kpopcontent)
            print('=========================================================')
            
    except Exception as ex: # 에러 종류
        print('에러가 발생 했습니다', ex) #
        driver.quit()
    driver.quit()

In [ ]:
kpop_url_list = drama_crawler_url()
# print(kpop_url_list)
with open('./k_drama_url_page_fron_1_to_6_0801.json', 'w', encoding='UTF-8') as outfile:
    json.dump(kpop_url_list, outfile, ensure_ascii=False)

In [ ]:
# k_drama_url_page_fron_1_to_6
with open('./k_drama_url_page_fron_1_to_6_0801.json', 'r') as file:
    json_data = json.load(file)
for title in json_data:
    url = json_data[title]['href']
    print(url)
    try:
        data = korea_drama_detail_page(url)
        
    except:
        print('다른 페이지 오류를 어떻게 잡지')
    

In [ ]:
['호텔 델루나']